In [3]:
import pandas as pd

def openFile(file_path, mode='r'):
    try:
        return open(file_path, mode)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise

fileGenre = openFile('../data/u.genre')
fileItem = openFile('../data/u.item')
fileUser = openFile('../data/u.user')
fileData = openFile('../data/u.data')

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        


        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gender'] = rec[2]
            item['function'] = rec[4]
        else:
            print("Invalid record format. Skipping.")
            return

        data.append(item)
    filed.close()
    return data

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)


df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)

In [3]:

fileData = open('../data/u.data', 'r')
def getDataframeFromFileWithSpaces(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

relationship = getDataframeFromFileWithSpaces(fileData)
df_relations = pd.DataFrame(relationship)

In [4]:
merged_df = pd.merge(df_movies, df_relations[['userId', 'movieId', 'rating','timestamp']], on='movieId', how='inner')

In [5]:
merged_df[(merged_df['movieId'] == '1682') & (merged_df['userId'] == '916')]

,movieId,name,release_date,url,genre,userId,rating,timestamp
99999,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,http://us.imdb.com/M/title-exact?Schrei%20aus%...,[Drama],916,3,880845755


<hr>
<p style='color:red;'> Note: The input data must be modeled in the following way :</p>
<code>
{"userId": "365",
  "movie": 
    { "movieId": "7526",
        "title": "Lord of the Rings: The Fellowship of the Ring, The (2001)",
        "genres": ["Adventure", "Fantasy"] 
    },
    "rating": "5",
    "timestamp": "45554125" 
}
</code>
<p style='color:red;'>  Ouput:</p>
<hr>

In [6]:
movie_list = []
for x in merged_df.itertuples():
    items = {"userId": x.userId,
        "movie": 
        { "movieId": x.movieId,
            "title": x.name,
            "genres": x.genre
        },
        "rating": x.rating,
        "timestamp": x.timestamp
    }
    movie_list.append(items)
df = pd.DataFrame(movie_list)

In [7]:
df.to_json(orient='records',path_or_buf="../data/output.json")

<p style='color:red;'> Create A flask Api :</p>
<code>
flask --app app --debug run
</code>

In [8]:
df = pd.read_json(orient='records',path_or_buf='../data/output.json')
print(df)

       userId                                              movie  rating   
0         308  {'movieId': '1', 'title': 'Toy Story (1995)', ...       4  \
1         287  {'movieId': '1', 'title': 'Toy Story (1995)', ...       5   
2         148  {'movieId': '1', 'title': 'Toy Story (1995)', ...       4   
3         280  {'movieId': '1', 'title': 'Toy Story (1995)', ...       4   
4          66  {'movieId': '1', 'title': 'Toy Story (1995)', ...       3   
...       ...                                                ...     ...   
99995     863  {'movieId': '1678', 'title': 'Mat' i syn (1997...       1   
99996     863  {'movieId': '1679', 'title': 'B. Monkey (1998)...       3   
99997     863  {'movieId': '1680', 'title': 'Sliding Doors (1...       2   
99998     896  {'movieId': '1681', 'title': 'You So Crazy (19...       3   
99999     916  {'movieId': '1682', 'title': 'Scream of Stone ...       3   

                timestamp  
0     1998-02-17 17:28:52  
1     1997-09-27 04:21:28  
2  